
***

<br>
Let us look at how we can implement text classification using PyTorch
<br>

We would implement text classification using a simple embedding bag of words using PyTorch on tweet data to classify tweets as "positive" or "negative"


<br>

***


In [1]:

import os
import re
import shutil
import string


from collections import Counter


import pandas as pd
import numpy as np

import sklearn


from sklearn.model_selection import train_test_split




****
Let us define methods to pre-process the review data
****

In [3]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_url(text): 
    url_pattern  = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    return url_pattern.sub(r'', text)
 # converting return value from list to string



def clean_text(text ): 
    delete_dict = {sp_character: '' for sp_character in string.punctuation} 
    delete_dict[' '] = ' ' 
    table = str.maketrans(delete_dict)
    text1 = text.translate(table)
    #print('cleaned:'+text1)
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr if ( not w.isdigit() and  ( not w.isdigit() and len(w)>2))]) 
    
    return text2.lower()



In [3]:
# def get_sentiment(sentiment):
#     if sentiment == 'positive':
#         return 2
#     elif sentiment == 'negative':
#         return 1
#     else:
#         return 0
    

In [5]:
train_data= pd.read_csv("cleaned/train.csv")
train_data.dropna(axis = 0, how ='any',inplace=True) 
train_data['Num_words_text'] = train_data['tweet'].apply(lambda x:len(str(x).split())) 
mask = train_data['Num_words_text'] >2
train_data = train_data[mask]
print('-------Train data--------')
print(train_data['sentiment'].value_counts())
print(len(train_data))
print('-------------------------')
max_train_sentence_length  = train_data['Num_words_text'].max()


train_data['tweet'] = train_data['tweet'].apply(remove_emoji)
train_data['tweet'] = train_data['tweet'].apply(remove_url)
train_data['tweet'] = train_data['tweet'].apply(clean_text)

train_data['label'] = train_data['sentiment']

test_data= pd.read_csv("cleaned/test.csv")
test_data.dropna(axis = 0, how ='any',inplace=True) 
test_data['Num_words_text'] = test_data['tweet'].apply(lambda x:len(str(x).split())) 

max_test_sentence_length  = test_data['Num_words_text'].max()

mask = test_data['Num_words_text'] >2
test_data = test_data[mask]

print('-------Test data--------')
print(test_data['sentiment'].value_counts())
print(len(test_data))
print('-------------------------')

test_data['tweet'] = test_data['tweet'].apply(remove_emoji)
test_data['tweet'] = test_data['tweet'].apply(remove_url)
test_data['tweet'] = test_data['tweet'].apply(clean_text)

test_data['label'] = test_data['sentiment']

print('Train Max Sentence Length :'+str(max_train_sentence_length))
print('Test Max Sentence Length :'+str(max_test_sentence_length))




-------Train data--------
1    478915
0    478199
Name: sentiment, dtype: int64
957114
-------------------------
-------Test data--------
1    287373
0    286674
Name: sentiment, dtype: int64
574047
-------------------------
Train Max Sentence Length :32
Test Max Sentence Length :34


In [11]:
train_data.head(10)

,Unnamed: 0,index,Unnamed: 0.1,sentiment,id,date,no_query,username,tweet,Num_words_text,label
0,0,93365,93365,0,1770614161,Mon May 11 21:47:43 PDT 2009,NO_QUERY,TheNostalgicOne,itsmemorphious thats terrible,3,0
1,1,1361704,1361704,1,2049164413,Fri Jun 05 16:24:27 PDT 2009,NO_QUERY,austen_addict,bookclubgirl delighted have entertained you,5,1
2,2,1314011,1314011,1,2013853646,Tue Jun 02 23:45:37 PDT 2009,NO_QUERY,TanyaMillerrr,katedangerous omg love puppies and thanks for ...,14,1
4,4,1334089,1334089,1,2016619108,Wed Jun 03 07:05:05 PDT 2009,NO_QUERY,Branflakes819,crystalmulligan dont worry its only presale,6,1
5,5,907080,907080,1,1695654133,Mon May 04 06:38:51 PDT 2009,NO_QUERY,southernlady75,absolutspacegrl could feel the excitement that...,16,1
6,6,227609,227609,0,1978095298,Sat May 30 22:25:48 PDT 2009,NO_QUERY,Tjb224,gunnerz20 iguess can see just not phone,7,0
7,7,409027,409027,0,2059548288,Sat Jun 06 17:09:14 PDT 2009,NO_QUERY,TaylorMarie2025,studying for final exams,4,0
8,8,1218727,1218727,1,1989863715,Mon Jun 01 03:04:31 PDT 2009,NO_QUERY,ZogeyBear,the way graves park see some animals then pub ...,14,1
9,9,340653,340653,0,2014884342,Wed Jun 03 03:00:21 PDT 2009,NO_QUERY,ssieg,heading into work this somewhat chilly and gra...,19,0
10,10,1403917,1403917,1,2054949796,Sat Jun 06 08:24:22 PDT 2009,NO_QUERY,hoopfeed,mickie1 bummer youre working sure the twitterv...,14,1


In [12]:
test_data.head(10)

,Unnamed: 0,index,Unnamed: 0.1,sentiment,id,date,no_query,username,tweet,Num_words_text,label
0,1000000,1189027,1189027,1,1983449528,Sun May 31 13:10:39 PDT 2009,NO_QUERY,marioc2010,lol especially you,3,1
1,1000001,1579780,1579780,1,2189985349,Tue Jun 16 01:26:42 PDT 2009,NO_QUERY,_rosieCAKES,xtndvla oooo likesssnow shall make sure thank ...,11,1
2,1000002,537873,537873,0,2198909374,Tue Jun 16 16:44:47 PDT 2009,NO_QUERY,lilhartman,back from from vegas and more vacations for wh...,11,0
3,1000003,1568502,1568502,1,2188227745,Mon Jun 15 21:31:11 PDT 2009,NO_QUERY,saduna,new handbag new true love,5,1
4,1000004,885394,885394,1,1686488157,Sun May 03 05:38:48 PDT 2009,NO_QUERY,helengop,cant wait for the bruins tonight with carolina,8,1
5,1000005,621398,621398,0,2228794175,Thu Jun 18 14:53:42 PDT 2009,NO_QUERY,xXxsKuLLzxXx,ipod frozen omg what shud cant live without music,9,0
6,1000006,230579,230579,0,1978770569,Sun May 31 00:24:38 PDT 2009,NO_QUERY,jennymurder,mom went the again tonight but justin snl and ...,13,0
7,1000007,1322847,1322847,1,2014836811,Wed Jun 03 02:51:01 PDT 2009,NO_QUERY,suddentwilight,vinod sharanya too had own fruit garden jamuns...,11,1
8,1000008,415919,415919,0,2061182622,Sat Jun 06 20:20:43 PDT 2009,NO_QUERY,shabrinasudiro,cant stand seeing anyone cryin,5,0
9,1000009,1146882,1146882,1,1977997349,Sat May 30 22:11:20 PDT 2009,NO_QUERY,emilayyy,havent watched degrassi forever but since cass...,13,1


***
Let us create our train,valid and test datasets
***

In [6]:
X_train, X_valid, Y_train, Y_valid= train_test_split(train_data['tweet'].tolist(),\
                                                      train_data['label'].tolist(),\
                                                      test_size=0.2,\
                                                      stratify = train_data['label'].tolist(),\
                                                      random_state=0)


print('Train data len:'+str(len(X_train)))
print('Class distribution'+str(Counter(Y_train)))


print('Valid data len:'+str(len(X_valid)))
print('Class distribution'+ str(Counter(Y_valid)))

print('Test data len:'+str(len(test_data['tweet'].tolist())))
print('Class distribution'+ str(Counter(test_data['label'].tolist())))


train_dat =list(zip(Y_train,X_train))
valid_dat =list(zip(Y_valid,X_valid))
test_dat=list(zip(test_data['label'].tolist(),test_data['tweet'].tolist()))


Train data len:765691
Class distributionCounter({1: 383132, 0: 382559})
Valid data len:191423
Class distributionCounter({1: 95783, 0: 95640})
Test data len:574047
Class distributionCounter({1: 287373, 0: 286674})


In [7]:
import torch
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

***
Let us create our vocabulary on train data
***

In [8]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')
train_iter = train_dat
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])


ModuleNotFoundError: No module named 'torchtext'

Let us define our text and label preprocessing pipleines

In [21]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) 

In [19]:
text_pipeline('here is the an example')


[61, 0, 1, 0, 4279]

In [20]:
label_pipeline('1')

1

Let us define our batch collation function

In [21]:


def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

#train_iter =train_dat
#dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

Let us define our text classification model

In [13]:
from torch import nn
import torch.nn.functional as F

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc1 = nn.Linear(embed_dim,64)
        self.fc2 = nn.Linear(64,16)
        self.fc3 = nn.Linear(16, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc1.weight.data.uniform_(-initrange, initrange)
        self.fc1.bias.data.zero_()
        self.fc2.weight.data.uniform_(-initrange, initrange)
        self.fc2.bias.data.zero_()
        self.fc3.weight.data.uniform_(-initrange, initrange)
        self.fc3.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        x = F.relu(self.fc1(embedded))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

Let us create an object of our text classification class

In [23]:
train_iter1 = train_dat
num_class = len(set([label for (label, text) in train_iter1]))
print(num_class)
vocab_size = len(vocab)
emsize = 128
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

2


Let us define our train and evaluate methods

In [24]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

Let us create dataloaders for text,train and validation data iterators and then train our model

In [25]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR =10  # learning rate
BATCH_SIZE = 16 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

train_iter2 = train_dat
test_iter2 =test_dat 
valid_iter2= valid_dat




train_dataloader = DataLoader(train_iter2, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(valid_iter2, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_iter2, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |   500/47856 batches | accuracy    0.499
| epoch   1 |  1000/47856 batches | accuracy    0.553
| epoch   1 |  1500/47856 batches | accuracy    0.617
| epoch   1 |  2000/47856 batches | accuracy    0.659
| epoch   1 |  2500/47856 batches | accuracy    0.678
| epoch   1 |  3000/47856 batches | accuracy    0.671
| epoch   1 |  3500/47856 batches | accuracy    0.668
| epoch   1 |  4000/47856 batches | accuracy    0.676
| epoch   1 |  4500/47856 batches | accuracy    0.691
| epoch   1 |  5000/47856 batches | accuracy    0.690
| epoch   1 |  5500/47856 batches | accuracy    0.704
| epoch   1 |  6000/47856 batches | accuracy    0.692
| epoch   1 |  6500/47856 batches | accuracy    0.704
| epoch   1 |  7000/47856 batches | accuracy    0.690
| epoch   1 |  7500/47856 batches | accuracy    0.693
| epoch   1 |  8000/47856 batches | accuracy    0.697
| epoch   1 |  8500/47856 batches | accuracy    0.700
| epoch   1 |  9000/47856 batches | accuracy    0.706
| epoch   1 |  9500/47856 ba

In [26]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.784


Test model on text

In [30]:
sentiment_label = {1: "Positive",0: "Negative"}

def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() 
ex_text_str = "kill die death"
model = model.to("cpu")

print("This is a %s tweet" %sentiment_label[predict(ex_text_str, text_pipeline)])

This is a Negative tweet


In [33]:
checkpoint = {'model': TextClassificationModel(vocab_size=len(vocab),embed_dim=128,num_class=2),
              'state_dict': model.state_dict(),
              'optimizer' : optimizer.state_dict()}

torch.save(checkpoint, '/kaggle/working/checkpoint.pth')

In [25]:
sentiment_label = {1: "Positive",0: "Negative"}
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.load_state_dict(checkpoint['state_dict'])
    for parameter in model.parameters():
        parameter.requires_grad = False
    
    model.eval()
    
    return model

In [23]:
loaded_model = load_checkpoint('checkpoint.pth')
print(loaded_model)

TextClassificationModel(
  (embedding): EmbeddingBag(476068, 128, mode=mean)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=2, bias=True)
)


In [24]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) 
def predict_new(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = loaded_model(text, torch.tensor([0]))
        return output.argmax(1).item() 
ex_text_str = "heaven is great to love"
loaded_model = loaded_model.to("cpu")

print("This is a %s tweet" %sentiment_label[predict_new(ex_text_str, text_pipeline)])

NameError: name 'vocab' is not defined

In [26]:
new_model = torch.load('new.pth')

In [27]:
new_model.eval()

TextClassificationModel(
  (embedding): EmbeddingBag(476068, 128, mode=mean)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=2, bias=True)
)

In [31]:
def predict_new(text):
    with torch.no_grad():
        text = torch.tensor(text)
        output = loaded_model(text, torch.tensor([0]))
        return output.argmax(1).item() 
ex_text_str = "heaven is great to love"
new_model = new_model.to("cpu")

In [32]:
predict_new("shitty game")

TypeError: new(): invalid data type 'str'